# Pulling TMDb Data Using *tmdbsimple* Wrapper

In an attempt to create my own Letterboxd Analytics dashboard, I need to have the data related to my watching activities and the films information. Of course the easy way is to directly use the Letterboxd API, however it is not for public use. I was lucky because Letterboxd still provide the users the ability to export our data, but that excludes the films information (other than the title and the release year). Because of that, I'm using the TMDb API ([Documentation](https://developers.themoviedb.org/3/getting-started/introduction)) instead, and I'm specifically using the Python wrapper called [tmdbsimple by celiao](https://github.com/celiao/tmdbsimple) because it's much simpler to use and to get familiar with it, I also referred to this post by [Adina Steinman.](https://adinasteinman.medium.com/navigating-my-first-api-the-tmdb-database-d8d2975b0df4)

My work flow are structured as follows:
1. Get familiar with the wrapper
2. Extract the film `titles` and `release_year` from my `watched.csv` file
3. Use `tmdb.Search` to search the films by the `titles` and `release_year`. Get the list of their `id`
4. Use the `tmdb.Movies` and `tmdb.TV` to get the films information (genres, languages, countries, runtime, vote count and vote average) by using their `id`.
5. Convert into dataframe and export as .csv file.
6. Get frequencies for genres, languages and countries. Convert into dataframe and export as .csv file.

## 1. Get familiar with the wrapper

I want to know how the wrapper works and to understand the TMDb documentation. 

In [4]:
# Connecting to the API
import tmdbsimple as tmdb

tmdb.API_KEY = '6c37d50741c7004f344af19bf3a67b10'

# Declare variable to use the Search and Discover function
search = tmdb.Search()
discover = tmdb.Discover()

In [5]:
# Search any movies filtered by the company 'Sony Pictures'
response1 = search.company(query='Sony Pictures')
response1['results'][:2]

[{'id': 82346,
  'logo_path': '/jqgK6CSkPrEsIv6Nk390JaBcXYF.png',
  'name': 'Sony Pictures',
  'origin_country': 'JP'},
 {'id': 34,
  'logo_path': '/GagSvqWlyPdkFHMfQ3pNq6ix9P.png',
  'name': 'Sony Pictures',
  'origin_country': 'US'}]

In [7]:
# Discover any movies filtered by the release year '2016'

response2 = discover.movie(year="2016")
for value in response2['results']:
    print(value['title'])
    
print("Number of results: ", len(response2['results']))

Naruto Shippuden the Movie
The Last: Naruto the Movie
Dragon Ball Z: Resurrection 'F'
Spider-Man 3
Harry Potter and the Prisoner of Azkaban
Spider-Man
Your Name.
The Amazing Spider-Man 2
Hacksaw Ridge
Harry Potter and the Half-Blood Prince
Harry Potter and the Deathly Hallows: Part 1
The Conjuring
Don't Breathe
The Lion King
Harry Potter and the Deathly Hallows: Part 2
Deadpool
Train to Busan
The Maze Runner
Jurassic World
A Silent Voice: The Movie
Number of results:  20


## 2. Extract the film `titles` and `release_year` from my `watched.csv` file

In [8]:
# Import watched.csv
import pandas as pd

watched_df = pd.read_csv ("watched.csv")
watched_df

,Date,Name,Year,Letterboxd URI
0,2020-11-11,Parasite,2019,https://boxd.it/hTha
1,2020-11-11,Big Fish,2003,https://boxd.it/2a3M
2,2020-11-14,The Queen's Gambit,2020,https://boxd.it/s1Ym
3,2021-05-03,The Imitation Game,2014,https://boxd.it/6aig
4,2021-05-03,The Danish Girl,2015,https://boxd.it/9Ohy
...,...,...,...,...
86,2021-09-09,Forrest Gump,1994,https://boxd.it/728
87,2021-09-09,My First Client,2019,https://boxd.it/lLYS
88,2021-09-15,The Autopsy of Jane Doe,2016,https://boxd.it/dUu2
89,2021-09-15,The Final Girls,2015,https://boxd.it/9gv2


In [9]:
# Extract the titles for each film into a list
title = watched_df.Name.tolist()
print(title)
print(len(title))

['Parasite', 'Big Fish', "The Queen's Gambit", 'The Imitation Game', 'The Danish Girl', 'Changeling', 'To the Bone', 'Sherlock: The Abominable Bride', 'Sherlock: The Final Problem', 'La La Land', 'Scott Pilgrim vs. the World', 'Shrek 2', 'Split', 'Black Swan', 'The Babadook', 'Carrie', 'The Breakfast Club', "To All the Boys I've Loved Before", 'Dead Poets Society', 'Tangled', 'Stuart Little', 'Groundhog Day', 'Matilda', 'Kingsman: The Secret Service', 'Spirited Away', 'The Theory of Everything', 'Room', 'A Silent Voice', 'Your Name.', 'Bridge to Terabithia', 'Whiplash', 'A Walk to Remember', 'Amanda Knox', 'Do You Like Brahms?', 'Hidden Figures', 'American Murder: The Family Next Door', 'KIL', 'Pekak', 'Istanbul Aku Datang!', 'Gold Diggers', 'Pulang', 'Gol & Gincu', 'Gol & Gincu Vol 2', 'Upin & Ipin Geng Pengembaraan Bermula', '29 Februari', 'The Con-Heartist', 'Tootsies & The Fake', 'Miss Hammurabi', 'The Pursuit of Happyness', 'The Truman Show', 'Miss Congeniality', 'Easy A', 'Forgot

In [10]:
# Extract the release year for each film into a list
released_year = watched_df.Year.tolist()
print(released_year)
print(len(released_year))

[2019, 2003, 2020, 2014, 2015, 2008, 2017, 2016, 2017, 2016, 2010, 2004, 2016, 2010, 2014, 1976, 1985, 2018, 1989, 2010, 1999, 1993, 1996, 2014, 2001, 2014, 2015, 2016, 2016, 2007, 2014, 2002, 2016, 2020, 2016, 2020, 2013, 2016, 2012, 2009, 2018, 2005, 2018, 2009, 2012, 2020, 2019, 2018, 2006, 1998, 2000, 2010, 2017, 2018, 2009, 2019, 2008, 2005, 2018, 2019, 2004, 2011, 2014, 2004, 2017, 1990, 1993, 2019, 2019, 1988, 2003, 1995, 2009, 2020, 2021, 1994, 2021, 2019, 2021, 2018, 2020, 2017, 2020, 2015, 1997, 2008, 1994, 2019, 2016, 2015, 2019]
91


## 3. Use tmdb.Search to search the films by the titles and release_year, then get the list of their id.

In [11]:
# total_result is to count the number of results
result_count = 0

for i,j in zip(title,released_year):
    response = search.movie(query= i, year=j)
    for value in response['results']:
        if value['title'] == i:              # This if statement is to exclude results with inaccurate match   
            print(value['title'],"  /  ",
                  value['release_date'],"  /  ",
                  value['id'])
            
            result_count+= 1
            break            # Only take first result to avoid redundant
            
print("Results count: ", result_count)

Parasite   /   2019-05-30   /   496243
Big Fish   /   2003-12-25   /   587
The Imitation Game   /   2014-11-14   /   205596
The Danish Girl   /   2015-01-01   /   306819
Changeling   /   2008-01-30   /   3580
To the Bone   /   2017-01-22   /   401104
Sherlock: The Abominable Bride   /   2016-01-01   /   379170
La La Land   /   2016-11-29   /   313369
Scott Pilgrim vs. the World   /   2010-08-12   /   22538
Shrek 2   /   2004-05-19   /   809
Split   /   2016-11-15   /   381288
Black Swan   /   2010-12-03   /   44214
The Babadook   /   2014-05-22   /   242224
Carrie   /   1976-11-03   /   7340
The Breakfast Club   /   1985-02-15   /   2108
To All the Boys I've Loved Before   /   2018-08-16   /   466282
Dead Poets Society   /   1989-06-02   /   207
Tangled   /   2010-11-24   /   38757
Stuart Little   /   1999-12-17   /   10137
Groundhog Day   /   1993-02-11   /   137
Matilda   /   1996-08-02   /   10830
Kingsman: The Secret Service   /   2014-12-13   /   207703
Spirited Away   /   2001-07

That looks fine. Now, let's get their `id` into a list, `movie_id`.

In [12]:
movie_id = [] # a variable to store a list of movies id
result_count=0

for i,j in zip(title,released_year):
    response = search.movie(query= i, year=j)
    for value in response['results']:
        if value['title'] == i:
            movie_id.append(value['id'])
            result_count+= 1
            break 
            
print(movie_id)
print("Results count: ", result_count)

[496243, 587, 205596, 306819, 3580, 401104, 379170, 313369, 22538, 809, 381288, 44214, 242224, 7340, 2108, 466282, 207, 38757, 10137, 137, 10830, 207703, 129, 266856, 264644, 372058, 1265, 244786, 10229, 411009, 381284, 743601, 203854, 414101, 146899, 34852, 538528, 249884, 548405, 67792, 134312, 771257, 648968, 1402, 37165, 1493, 37735, 488623, 471506, 21208, 572164, 4922, 4348, 402900, 568091, 12153, 44826, 282761, 9833, 462477, 10166, 2758, 530385, 522518, 12477, 245439, 9603, 14836, 9587, 799555, 571787, 337404, 493922, 683287, 432836, 664277, 358329, 1624, 8390, 13, 591322, 397243, 293970, 458305]
Results count:  84


Notice that we should have 91 results instead of 84. The remaining 7 titles do  not gives any search result probably because it does not exist in the database, or it is a tv show instead of a movie (since I used `search.movie`). So, let's try to separate the ones with 0 results as tv_shows.

In [13]:
# zero_search is to count the number of results with 0 results
zero_search = 0

tv_title = [] # to store list of titles with 0 results
tv_year = []  # to store list of release year with 0 results

for i,j in zip(title,released_year):
    response = search.movie(query=i, year=j)
    if (len(response['results'])) == 0:
        tv_title.append(i)
        tv_year.append(j)
        print(i,j, len(response['results']))
        zero_search+= 1
        
print("0 search: ", zero_search)
print(tv_title)
print(tv_year)

The Queen's Gambit 2020 0
Sherlock: The Final Problem 2017 0
Do You Like Brahms? 2020 0
Miss Hammurabi 2018 0
Mr. Queen 2020 0
Crime Scene: The Vanishing at the Cecil Hotel 2021 0
0 search:  6
["The Queen's Gambit", 'Sherlock: The Final Problem', 'Do You Like Brahms?', 'Miss Hammurabi', 'Mr. Queen', 'Crime Scene: The Vanishing at the Cecil Hotel']
[2020, 2017, 2020, 2018, 2020, 2021]


Now I'm going to use `search.tv` to search for any results.

In [14]:
total = 0 # to count number of results
tv_id = []

for i,j in zip(tv_title,tv_year):
    response = search.tv(query= i, year=j)
    for value in response['results']:
        if value['name'] == i:
            tv_id.append(value['id'])
            total+= 1
            
print(tv_id)
print("Results count: ", total)

[87739, 101857, 79342, 108261, 116989]
Results count:  5


I only get 5 results instead of 6, but I'm going to deal with that later when cleaning data.

## 4. Use the tmdb.Movies and tmdb.TV to get the films information by using their ids.
Information to extract: genres, languages, countries, runtime, vote count and vote average
Lets try one id for each movie and tv show. <br>
(Movie: 496243=Parasite, TV: 116989 = Crime Scene: The Vanishing at the Cecil Hotel)

In [16]:
movie = tmdb.Movies(496243)
response = movie.info()
movie.title

'Parasite'

In [15]:
tv = tmdb.TV(116989)
response = tv.info()
tv.name

'Crime Scene: The Vanishing at the Cecil Hotel'

That works fine. Now let's get those data.

In [19]:
movie_info=[]

for i in movie_id:
    movie = tmdb.Movies(i)
    response = movie.info()
    movie_info.append([movie.id, movie.title, movie.spoken_languages, movie.production_countries, movie.genres, 
                       movie.runtime, movie.release_date, movie.vote_count, movie.vote_average])
print(len(movie_info))
movie_info[:2]

84


[[496243,
  'Parasite',
  [{'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'},
   {'english_name': 'German', 'iso_639_1': 'de', 'name': 'Deutsch'},
   {'english_name': 'Korean', 'iso_639_1': 'ko', 'name': '한국어/조선말'}],
  [{'iso_3166_1': 'KR', 'name': 'South Korea'}],
  [{'id': 35, 'name': 'Comedy'},
   {'id': 53, 'name': 'Thriller'},
   {'id': 18, 'name': 'Drama'}],
  133,
  '2019-05-30',
  12182,
  8.5],
 [587,
  'Big Fish',
  [{'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'}],
  [{'iso_3166_1': 'US', 'name': 'United States of America'}],
  [{'id': 12, 'name': 'Adventure'},
   {'id': 14, 'name': 'Fantasy'},
   {'id': 18, 'name': 'Drama'}],
  125,
  '2003-12-25',
  5737,
  7.8]]

In [17]:
tv_info=[]

for i in tv_id:
    tv = tmdb.TV(i)
    response = tv.info()
    tv_info.append([tv.id, tv.name, tv.origin_country, tv.genres, 
                   tv.spoken_languages, tv.episode_run_time,
                   tv.first_air_date, tv.number_of_seasons, 
                   tv.number_of_episodes, tv.vote_count, tv.vote_average])
tv_info

[[87739,
  "The Queen's Gambit",
  ['US'],
  [{'id': 18, 'name': 'Drama'}],
  [{'english_name': 'English', 'iso_639_1': 'en', 'name': 'English'}],
  [60],
  '2020-10-23',
  1,
  7,
  2185,
  8.7],
 [101857,
  'Do You Like Brahms?',
  ['KR'],
  [{'id': 18, 'name': 'Drama'}],
  [{'english_name': 'Korean', 'iso_639_1': 'ko', 'name': '한국어/조선말'}],
  [60],
  '2020-08-31',
  1,
  16,
  9,
  8.1],
 [79342,
  'Miss Hammurabi',
  ['KR'],
  [{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}],
  [{'english_name': 'Korean', 'iso_639_1': 'ko', 'name': '한국어/조선말'}],
  [80],
  '2018-05-21',
  1,
  16,
  7,
  8.4],
 [108261,
  'Mr. Queen',
  ['KR'],
  [{'id': 18, 'name': 'Drama'},
   {'id': 35, 'name': 'Comedy'},
   {'id': 10765, 'name': 'Sci-Fi & Fantasy'}],
  [{'english_name': 'Korean', 'iso_639_1': 'ko', 'name': '한국어/조선말'}],
  [72],
  '2020-12-12',
  1,
  20,
  181,
  8.8],
 [116989,
  'Crime Scene: The Vanishing at the Cecil Hotel',
  ['US'],
  [{'id': 99, 'name': 'Documentary'},
   {'id': 8

## 5. Convert into dataframe and export as .csv file.

In [20]:
movie_df = pd.DataFrame(movie_info, columns =['ID', 'title', 'languages', 'prod_country', 'genres', 'runtime', 'releasedate', 'votecount', 'voteavg'])
movie_df.head()

,ID,title,languages,prod_country,genres,runtime,releasedate,votecount,voteavg
0,496243,Parasite,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",133,2019-05-30,12182,8.5
1,587,Big Fish,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",125,2003-12-25,5737,7.8
2,205596,The Imitation Game,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 36, 'name': 'History'}, {'id': 18, 'na...",113,2014-11-14,13862,8.0
3,306819,The Danish Girl,"[{'english_name': 'German', 'iso_639_1': 'de',...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...","[{'id': 18, 'name': 'Drama'}]",119,2015-01-01,4964,7.6
4,3580,Changeling,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",141,2008-01-30,3180,7.6


In [18]:
# Convert it into dataframe
tv_df = pd.DataFrame(tv_info, columns =['ID', 'title', 'origin_country', 'genres', 'languages', 'eps_runtime', '1st_airdate', 'no_seasons', 'no_eps', 'vote_counts', 'vote_average'])
tv_df

,ID,title,origin_country,genres,languages,eps_runtime,1st_airdate,no_seasons,no_eps,vote_counts,vote_average
0,87739,The Queen's Gambit,[US],"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'English', 'iso_639_1': 'en'...",[60],2020-10-23,1,7,2185,8.7
1,101857,Do You Like Brahms?,[KR],"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'Korean', 'iso_639_1': 'ko',...",[60],2020-08-31,1,16,9,8.1
2,79342,Miss Hammurabi,[KR],"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'Korean', 'iso_639_1': 'ko',...",[80],2018-05-21,1,16,7,8.4
3,108261,Mr. Queen,[KR],"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'Korean', 'iso_639_1': 'ko',...",[72],2020-12-12,1,20,181,8.8
4,116989,Crime Scene: The Vanishing at the Cecil Hotel,[US],"[{'id': 99, 'name': 'Documentary'}, {'id': 80,...","[{'english_name': 'English', 'iso_639_1': 'en'...",[54],2021-02-10,1,4,211,7.3


Now do the same with movies.

In [21]:
# Export the dataframes into csv
movie_df.to_csv('watchedmovie.csv')
print("Exported movies successfully!")

tv_df.to_csv('watchedtv.csv')
print("Exported tv shows successfully!")

Exported movies successfully!
Exported tv shows successfully!


## 6. Get frequencies for genres, languages and countries.
These information has one or more data for one film, for example, the movie Parasite has 3 genres which are Comedy, Thriller and Drama. To me, it's easier to find the frequency using Python instead of Excel later. So I figured I'll just create separate csv files on their frequencies. Honestly I don't know other ways to do it for the time being :(. By the way, I got this solution from this [Kaggle kernel](https://www.kaggle.com/hassanshahin/genres-analysis-eda-clustring).

In [1]:
# Import watchedmovie.csv and watchedtv.csv
# Can skip this step if previous codes are still running
import pandas as pd

movie = pd.read_csv ("watchedmovie.csv")
movie.head()

,Unnamed: 0,ID,title,languages,prod_country,genres,runtime,releasedate,votecount,voteavg
0,0,496243,Parasite,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'KR', 'name': 'South Korea'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 53, 'nam...",133,2019-05-30,12182,8.5
1,1,587,Big Fish,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",125,2003-12-25,5737,7.8
2,2,205596,The Imitation Game,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'id': 36, 'name': 'History'}, {'id': 18, 'na...",113,2014-11-14,13862,8.0
3,3,306819,The Danish Girl,"[{'english_name': 'German', 'iso_639_1': 'de',...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...","[{'id': 18, 'name': 'Drama'}]",119,2015-01-01,4964,7.6
4,4,3580,Changeling,"[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",141,2008-01-30,3180,7.6


In [4]:
tv = pd.read_csv ("watchedtv.csv")
tv.head()

,Unnamed: 0,ID,title,origin_country,genres,languages,eps_runtime,1st_airdate,no_seasons,no_eps,vote_counts,vote_average
0,0,87739,The Queen's Gambit,['US'],"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'English', 'iso_639_1': 'en'...",[60],2020-10-23,1,7,2185,8.7
1,1,101857,Do You Like Brahms?,['KR'],"[{'id': 18, 'name': 'Drama'}]","[{'english_name': 'Korean', 'iso_639_1': 'ko',...",[60],2020-08-31,1,16,9,8.1
2,2,79342,Miss Hammurabi,['KR'],"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'Korean', 'iso_639_1': 'ko',...",[80],2018-05-21,1,16,7,8.4
3,3,108261,Mr. Queen,['KR'],"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'Korean', 'iso_639_1': 'ko',...",[72],2020-12-12,1,20,181,8.8
4,4,116989,Crime Scene: The Vanishing at the Cecil Hotel,['US'],"[{'id': 99, 'name': 'Documentary'}, {'id': 80,...","[{'english_name': 'English', 'iso_639_1': 'en'...",[54],2021-02-10,1,4,211,7.3


### 6.1 Frequencies for genres

In [2]:
from collections import Counter
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [3]:
movie['genres'] = movie['genres'].apply(convert)

genres = Counter()

for i in range(movie.shape[0]):
    for j in movie.genres[i]:
        genres[j]+=1
        
genres_df = pd.DataFrame.from_dict(genres, orient='index').reset_index()
genres_df = genres_df.rename(columns = {'index': 'Genres', 0: 'Frequency'})

print(genres_df.shape)
genres_df.head()

(18, 2)


,Genres,Frequency
0,Comedy,32
1,Thriller,13
2,Drama,46
3,Adventure,7
4,Fantasy,11


In [6]:
tv['genres'] = tv['genres'].apply(convert)

genres2 = Counter()

for i in range(tv.shape[0]):
    for j in tv.genres[i]:
        genres2[j]+=1
        
genres2_df = pd.DataFrame.from_dict(genres2, orient='index').reset_index()
genres2_df = genres2_df.rename(columns = {'index': 'Genres', 0: 'Frequency'})

print(genres2_df.shape)
genres2_df.head()

(6, 2)


,Genres,Frequency
0,Drama,4
1,Comedy,2
2,Sci-Fi & Fantasy,1
3,Documentary,1
4,Crime,1


#### Merging both dataframe using outer join

In [7]:
merged_genre = pd.merge(genres_df, genres2_df, on='Genres',how='outer')
merged_genre

,Genres,Frequency_x,Frequency_y
0,Comedy,32.0,2.0
1,Thriller,13.0,NaN
2,Drama,46.0,4.0
3,Adventure,7.0,NaN
4,Fantasy,11.0,NaN
5,History,3.0,NaN
6,War,2.0,NaN
7,Crime,12.0,1.0
8,Mystery,10.0,1.0
9,TV Movie,1.0,NaN


#### Replace NaN with 0

In [9]:
merged_genre.isna().sum()

Genres          0
Frequency_x     1
Frequency_y    13
dtype: int64

In [10]:
merged_genre = merged_genre.fillna(0)
merged_genre

,Genres,Frequency_x,Frequency_y
0,Comedy,32.0,2.0
1,Thriller,13.0,0.0
2,Drama,46.0,4.0
3,Adventure,7.0,0.0
4,Fantasy,11.0,0.0
5,History,3.0,0.0
6,War,2.0,0.0
7,Crime,12.0,1.0
8,Mystery,10.0,1.0
9,TV Movie,1.0,0.0


#### Create a new column with the aggregated values and drop Frequency_x and Frequency_y columns

In [12]:
merged_genre['Frequency'] = merged_genre['Frequency_x']+merged_genre['Frequency_y']
merged_genre = merged_genre.drop(["Frequency_x","Frequency_y"], axis=1)
merged_genre

,Genres,Frequency
0,Comedy,34.0
1,Thriller,13.0
2,Drama,50.0
3,Adventure,7.0
4,Fantasy,11.0
5,History,3.0
6,War,2.0
7,Crime,13.0
8,Mystery,11.0
9,TV Movie,1.0


In [13]:
merged_genre.to_csv('genre_freq.csv')
print("Exported successfully!")

Exported successfully!


### 6.2 Frequencies for languages

In [14]:
def convert_lang(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['english_name'])
    return L

In [33]:
movie['languages'] = movie['languages'].apply(convert_lang)

languages = Counter()

for i in range(movie.shape[0]):
    for j in movie.languages[i]:
        languages[j]+=1
        
lang_df = pd.DataFrame.from_dict(languages, orient='index').reset_index()
lang_df = lang_df.rename(columns = {'index': 'Languages', 0: 'Frequency'})

print(lang_df.shape)
lang_df.head()

(16, 2)


,Languages,Frequency
0,English,64
1,German,3
2,Korean,6
3,French,6
4,Italian,5


In [16]:
tv['languages'] = tv['languages'].apply(convert_lang)

languages2 = Counter()

for i in range(tv.shape[0]):
    for j in tv.languages[i]:
        languages2[j]+=1
        
lang2_df = pd.DataFrame.from_dict(languages2, orient='index').reset_index()
lang2_df = lang2_df.rename(columns = {'index': 'Languages', 0: 'Frequency'})

print(lang2_df.shape)
lang2_df.head()

(3, 2)


,Languages,Frequency
0,English,2
1,Korean,3
2,Spanish,1


In [17]:
merged_lang = pd.merge(lang_df, lang2_df, on='Languages',how='outer')
merged_lang

,Languages,Frequency_x,Frequency_y
0,English,64,2.0
1,German,3,NaN
2,Korean,6,3.0
3,French,6,NaN
4,Italian,5,NaN
5,Japanese,3,NaN
6,Latin,1,NaN
7,Malay,9,NaN
8,Mandarin,2,NaN
9,Thai,3,NaN


In [18]:
merged_lang = merged_lang.fillna(0)
merged_lang['Frequency'] = merged_lang['Frequency_x']+merged_lang['Frequency_y']
merged_lang = merged_lang.drop(["Frequency_x","Frequency_y"], axis=1)
merged_lang

,Languages,Frequency
0,English,66.0
1,German,3.0
2,Korean,9.0
3,French,6.0
4,Italian,5.0
5,Japanese,3.0
6,Latin,1.0
7,Malay,9.0
8,Mandarin,2.0
9,Thai,3.0


In [19]:
merged_lang.to_csv('language_freq.csv')
print("Exported successfully!")

Exported successfully!


### 6.3 Frequencies for countries

In [32]:
movie['prod_country'] = movie['prod_country'].apply(convert)

country = Counter()

for i in range(movie.shape[0]):
    for j in movie.prod_country[i]:
        country[j]+=1
        
country_df = pd.DataFrame.from_dict(country, orient='index').reset_index()
country_df = country_df.rename(columns = {'index': 'Countries', 0: 'Frequency'})

print(country_df.shape)
country_df.head()

(19, 2)


,Countries,Frequency
0,South Korea,6
1,United States of America,55
2,United Kingdom,14
3,Germany,4
4,Belgium,2


In [34]:
tv['origin_country']=tv['origin_country'].str.strip("[']")
tv

,Unnamed: 0,ID,title,origin_country,genres,languages,eps_runtime,1st_airdate,no_seasons,no_eps,vote_counts,vote_average
0,0,87739,The Queen's Gambit,US,[Drama],[English],[60],2020-10-23,1,7,2185,8.7
1,1,101857,Do You Like Brahms?,KR,[Drama],[Korean],[60],2020-08-31,1,16,9,8.1
2,2,79342,Miss Hammurabi,KR,"[Drama, Comedy]",[Korean],[80],2018-05-21,1,16,7,8.4
3,3,108261,Mr. Queen,KR,"[Drama, Comedy, Sci-Fi & Fantasy]",[Korean],[72],2020-12-12,1,20,181,8.8
4,4,116989,Crime Scene: The Vanishing at the Cecil Hotel,US,"[Documentary, Crime, Mystery]","[English, Spanish]",[54],2021-02-10,1,4,211,7.3


In [35]:
country2 = Counter()

for i in range(tv.shape[0]):
    for j in tv.origin_country:
        country2[j]+=1
        
country2_df = pd.DataFrame.from_dict(country2, orient='index').reset_index()
country2_df = country2_df.rename(columns = {'index': 'Countries', 0: 'Frequency'})

print(country2_df.shape)
country2_df.head()

(2, 2)


,Countries,Frequency
0,US,10
1,KR,15


In [36]:
# merge dataframes
merged_country = pd.merge(country_df, country2_df, on='Countries',how='outer')

# replace missing values with 0
merged_country = merged_country.fillna(0)

# create a new column with aggregated value and drop columns Frequency_x and Frequency_y
merged_country['Frequency'] = merged_country['Frequency_x']+merged_country['Frequency_y']
merged_country = merged_country.drop(["Frequency_x","Frequency_y"], axis=1)
merged_country

,Countries,Frequency
0,South Korea,6.0
1,United States of America,55.0
2,United Kingdom,14.0
3,Germany,4.0
4,Belgium,2.0
5,Denmark,3.0
6,Canada,4.0
7,Japan,4.0
8,Australia,1.0
9,Ireland,3.0


In [37]:
merged_country.to_csv('country_freq.csv')
print("Exported successfully!")

Exported successfully!


The end